### Data Generator

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [2]:
train_idg = image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True
)

val_idg = image.ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_generator = train_idg.flow_from_directory(
    '../Dataset/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 273 images belonging to 2 classes.


In [4]:
val_generator = val_idg.flow_from_directory(
    '../Dataset/test',
    target_size=(224,224),
    batch_size=24,
    class_mode='binary'
)

Found 68 images belonging to 2 classes.


### Scratch Model

In [5]:
from keras.layers import *
from keras.models import Sequential

In [6]:
model = Sequential()

In [7]:
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [9]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

- Train Model

In [10]:
from keras.callbacks import ModelCheckpoint
import os

In [11]:
if not os.path.exists('../Weights/Scratch'):
    os.mkdir('../Weights/Scratch')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [12]:
import os
import glob

files = glob.glob('../Weights/Scratch/*')
for f in files:
    os.remove(f)

In [13]:
checkpoint = ModelCheckpoint('../Weights/Scratch/model.{epoch}_{val_accuracy:.2f}.h5', 
                             monitor='val_accuracy', 
                             verbose=1, 
                             mode='auto',
                             save_weights_only=True
                            )

In [14]:
hist = model.fit(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data=val_generator,
    validation_steps=2,
    callbacks=checkpoint
)

Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 1.2789 - accuracy: 0.5714 
Epoch 00001: saving model to ../Weights/Scratch/model.1_0.40.h5
9/9 [==============================] - 120s 13s/step - loss: 1.2789 - accuracy: 0.5714 - val_loss: 0.6932 - val_accuracy: 0.3958
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 0.6605 - accuracy: 0.5751 
Epoch 00002: saving model to ../Weights/Scratch/model.2_0.56.h5
9/9 [==============================] - 112s 12s/step - loss: 0.6605 - accuracy: 0.5751 - val_loss: 0.6486 - val_accuracy: 0.5625
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 0.4744 - accuracy: 0.8059 
Epoch 00003: saving model to ../Weights/Scratch/model.3_0.79.h5
9/9 [==============================] - 108s 12s/step - loss: 0.4744 - accuracy: 0.8059 - val_loss: 0.4436 - val_accuracy: 0.7917
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 0.2606 - accuracy: 0.8938 
Epoch 00004: saving model to ../Weights/Scratch/m

In [20]:
x,y=val_generator.next()

In [21]:
model.evaluate(x,y)

1/1 [==============================] - 0s 0s/step - loss: 0.2078 - accuracy: 1.0000


[0.20783381164073944, 1.0]

- Save Model History

In [17]:
import numpy as np
import os

In [18]:
if not os.path.exists('../Model_history'):
    os.mkdir('../Model_history')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [19]:
np.save('../Model_history/scratch_file.npy',hist.history)
read_dictionary = np.load('../Model_history/scratch_file.npy',allow_pickle='TRUE').item()
print(read_dictionary.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
